In [1]:
import numpy_financial as npf
from typing import List

## Q3.1

我们可以使用[numpy-financial](https://numpy.org/numpy-financial/latest/index.html)库来直接计算现金流的IRR。这里再实现一个简单的`checkFlow()`函数检查一下现金流在给定IRR下是否等于现值PV。

In [2]:
def solvePV(flow: List[float], when: List[float], IRR: float) -> float:
    """Solve the PV of a given cash flow.

    Args:
        flow[N]: a cash flow
        when[N]: timeline of the cash flow
    
    Return:
        PV: PV of the given flow
    """
    assert len(flow) == len(when), "Invalid Cash Flow: {flow} with its time line {when}."

    PV = 0.
    for cash, time in zip(flow, when):
        PV += cash / (1 + IRR)**time

    return PV

小红办卡后每次理发的价值为`500`元(尽管她的实际支付为`400`元)，因此可以得到她办卡后的现金流为：

1. 当天支出`1200`元办卡，得到`500`元服务
2. 明年理发得到`500`元服务
3. 后年理发得到`500`元服务

因此IRR满足的方程为：

$$
1200 = 500 + \frac{500}{1 + y} + \frac{500}{(1 + y)^2}
$$

通过计算可以得到办卡的IRR为：

In [3]:
IRR = npf.irr([-700, 500, 500])
print(f"The IRR of this membership card is {IRR:.2%}.")

The IRR of this membership card is 27.47%.


最后简单验算一下三次理发的现值PV是否为`1200`元.

In [4]:
print(f"The PV is {solvePV([500, 500, 500], [0, 1, 2], IRR)}")

The PV is 1200.0


## Q3.2

(a) 使用票息剥离法(bootstrap method)计算三种债券的即期收益率(spot rate)。1年期国债的即期收益率$r_1$为：

$$
90 = \frac{100}{1 + r_1}, \ \ \ r_1 = 11.11 \%
$$

接下来计算2年期国债的即期收益率$r_2$为：

$$
95 = \frac{5}{1 + r_1} + \frac{105}{{(1 + r_2)}^2}, \ \ \ r_2 = 7.71 \%
$$

类似地，3年期国债的即期收益率$r_3$为：

$$
96 = \frac{6}{1 + r_1} + \frac{6}{{(1 + r_2)}^2} + \frac{106}{{(1 + r_3)}^3}, \ \ \ r_3 = 7.46 \%
$$

In [5]:
r1 = 100 / 90 - 1
r2 = (105 / (95 - 5 / (1 + r1)))**(1/2) - 1
r3 = (106 / (96 - 6 / (1 + r1) - 6 / (1 + r2)**2))**(1/3) - 1

print(f"r1 = {r1:.2%}, r2 = {r2:.2%}, r3 = {r3:.2%}")

r1 = 11.11%, r2 = 7.71%, r3 = 7.46%


(b) 到期收益率可以按照现金流来计算，三种债券分别满足方程：

$$
90 = \frac{1}{1 + y_1}
$$

$$
95 = \frac{5}{1 + y_2} + \frac{105}{(1 + y_2)^2}
$$

$$
96 = \frac{6}{1 + y_3} + \frac{6}{(1 + y_3)^2} + \frac{106}{(1 + y_3)^3}
$$

通过计算得到到期收益率分别为：

$$
y_1 = 11.11\%, \ \ \ y_2 = 7.80\%, \ \ \ y_3 = 7.54\%
$$

In [6]:
y1 = npf.irr([-90, 100])
y2 = npf.irr([-95, 5, 105])
y3 = npf.irr([-96, 6, 6, 106])

print(f"y1 = {y1:.2%}, y2 = {y2:.2%}, y3 = {y3:.2%}")

y1 = 11.11%, y2 = 7.80%, y3 = 7.54%


(c) 1年后的远期1年期即期利率满足方程：

$$
95 = \frac{5}{1 + r_1} + \frac{105}{(1 + r_1)(1 + fr)}, \ \ \ fr = 4.42\%
$$

In [7]:
fr = 105 / ((1+r1) * (95 - 5 / (1+r1))) - 1
print(f"fr = {fr:.2%}")

fr = 4.42%


类似地，2年后的远期1年期即期利率满足方程：

$$
96 = \frac{6}{1 + r_1} + \frac{6}{(1 + r_2)^2} + \frac{106}{(1 + r_2)^2 \cdot (1 + fr)}, \ \ \ fr = 6.95\%
$$

In [8]:
fr = 106 / ((1+r2)**2 * (96 - 6/(1+r1) - 6/(1+r2)**2)) - 1
print(f"fr = {fr:.2%}")

fr = 6.95%


(d) 1年后的远期2年期即期利率满足方程：

$$
96 = \frac{6}{1 + r_1} + \frac{6}{(1 + r_2)^2} + \frac{106}{(1 + r_1) \cdot (1 + fr)^2}, \ \ \ fr = 5.68\%
$$

In [9]:
fr = (106 / ((1+r1) * (96 - 6/(1+r1) - 6/(1+r2)**2)))**(1/2) - 1
print(f"fr = {fr:.2%}")

fr = 5.68%


(e) 该债券现在的价格满足方程

$$
P = \frac{10}{1 + r_1} + \frac{10}{(1 + r_2)^2} + \frac{110}{(1+r_3)^3} = 106.27
$$

In [10]:
P = 10/(1 + r1) + 10/(1 + r2)**2 + 110/(1 + r3)**3
print(f"P = {P:.2f}")

P = 106.27


## Q3.3

投资者在第2年通过购买债券获得的资金为：

$$
5 \times (1 + 10\%) + 105 = 110.5
$$

而融资的借款在偿还时需要支付

$$
97 \times (1 + 5.26\%) \times (1 + 10\%) = 112.31
$$

因此如果1年后的1年期即期利率升到了10%，投资者通过收益率赌博亏了 1.81 元。